In [15]:
from utils.eda import *
# Set default config for plotly show:
config = {'displayModeBar': False}

In [36]:
df = load_sample(size=1_000)
df.head(5)

Sample file found. Loading sample file...


,ID,parent,created,lifeSpan,speed,maxEnergy,kidEnergy,sensors,ancestor,nkids,pgmDeath
0,27309963,13,19766,5,1.0,3,3,5,27309963,3,20
1,20866257,43,15081,2,2.0,2,3,5,20866257,1,20
2,21062395,20,15224,3,1.0,5,2,5,21062395,1,19
3,29020663,46,21011,3,1.0,3,2,5,29020663,1,20
4,16046846,31,11562,2,2.0,2,2,5,16046846,1,20


In [17]:
quantVars = ["lifeSpan", "speed", "maxEnergy", "kidEnergy", "sensors", "nkids", "pgmDeath"]

In [18]:
ridgeline(df, quantVars).show(config=config)

In [19]:
scatter_3d(df,
    x='created',
    y='maxEnergy',
    z='lifeSpan',
    color='sensors'
    ).show(config=config)

In [46]:
x='created'
y='maxEnergy'
z='lifeSpan'

node_df = df.set_index('ID')
node_dict = node_df.to_dict(orient='index')
edges = df[['parent', 'ID']].dropna().rename(columns={ 'parent':'source', 'ID':'target'})
for e in ['source', 'target']:
    for axis in [x,y,z]:
        edges[f'{e}_{axis}'] = [node_dict[id][axis] for id in edges[e]]


,source,target
0,13,27309963
1,43,20866257
2,20,21062395
3,46,29020663
4,31,16046846
...,...,...
995,30,15029738
996,5,13791038
997,24,23830931
998,41,7068262


In [30]:
edgelist = df[['parent', 'ID']].dropna()
g = nx.from_pandas_edgelist(edgelist, source='parent', target='ID')
# set node positions:
pos = nx.spring_layout(g, dim=3)
Xn = [pos[k][0] for k in g.nodes]
Yn = [pos[k][1] for k in g.nodes]
Zn = [pos[k][2] for k in g.nodes]
# set edge positions:
Xe = []
Ye = []
Ze = []
for e in g.edges:
    Xe += [pos[e[0]][0], pos[e[1]][0], None]
    Ye += [pos[e[0]][1], pos[e[1]][1], None]
    Ze += [pos[e[0]][2], pos[e[1]][2], None]
# create edge trace
edge_trace = go.Scatter3d(x=Xe, y=Ye, z=Ze, mode='lines', line=dict(color='white', width=1), hoverinfo='none')
# create node trace
node_trace = go.Scatter3d(x=Xn, y=Yn, z=Zn, mode='markers', name='nodes', marker=dict(symbol='circle', size=6, color='blue', line=dict(color='rgb(50,50,50)', width=0.5)), hoverinfo='text')
# create figure
fig = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(title='Graph', showlegend=False, scene=dict(xaxis=dict(title='X', showgrid=False, zeroline=False, showline=False, showticklabels=False), yaxis=dict(title='Y', showgrid=False, zeroline=False, showline=False, showticklabels=False), zaxis=dict(title='Z', showgrid=False, zeroline=False, showline=False, showticklabels=False)), margin=dict(t=100), hovermode='closest'))

In [31]:
fig.show()